<a href="https://colab.research.google.com/github/Dhanashree-31/Dhanashree-31/blob/main/eyrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

'''
*****************************************************************************************
*
*        		===============================================
*           		GeoGuide(GG) Theme (eYRC 2023-24)
*        		===============================================
*
*  This script is to implement Task 1A of GeoGuide(GG) Theme (eYRC 2023-24).
*
*  This software is made available on an "AS IS WHERE IS BASIS".
*  Licensee/end user indemnifies and will keep e-Yantra indemnified from
*  any and all claim(s) that emanate from the use of the Software or
*  breach of the terms of this agreement.
*
*****************************************************************************************
'''

# Team ID:			[ GG_1673 ]
# Author List:		[ Vedant Adulkar, Dhanashree Pawar, Atharva Kocharekar, Mohammed Hazique Sayed]
# Filename:			task_1a.py
# Functions:	    [`ideantify_features_and_targets`, `load_as_tensors`,
# 					 `model_loss_function`, `model_optimizer`, `model_number_of_epochs`, `training_function`,
# 					 `validation_functions` ]

####################### IMPORT MODULES #######################
import pandas
import torch
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

###################### Additional Imports ####################
'''
You can import any additional modules that you require from
torch, matplotlib or sklearn.
You are NOT allowed to import any other libraries. It will
cause errors while running the executable
'''


##############################################################

################# ADD UTILITY FUNCTIONS HERE #################


##############################################################

def data_preprocessing(task_1a_dataframe):
    '''
    Purpose:
    ---
    This function will be used to load your csv dataset and preprocess it.
    Preprocessing involves cleaning the dataset by removing unwanted features,
    decision about what needs to be done with missing values etc. Note that
    there are features in the csv file whose values are textual (eg: Industry,
    Education Level etc)These features might be required for training the model
    but can not be given directly as strings for training. Hence this function
    should return encoded dataframe in which all the textual features are
    numerically labeled.

    Input Arguments:
    ---
    `task_1a_dataframe`: [Dataframe]
                          Pandas dataframe read from the provided dataset

    Returns:
    ---
    `encoded_dataframe` : [ Dataframe ]
                          Pandas dataframe that has all the features mapped to
                          numbers starting from zero

    Example call:
    ---
    encoded_dataframe = data_preprocessing(task_1a_dataframe)
    '''

    #################	ADD YOUR CODE HERE	##################
    task_1a_dataframe['Education'] = LabelEncoder().fit_transform(task_1a_dataframe['Education'])
    task_1a_dataframe
    encoded_dataframe = task_1a_dataframe
    encoded_dataframe['EverBenched'] = encoded_dataframe['EverBenched'].apply(lambda x: 1 if x == 'Yes' else 0)
    encoded_dataframe['Gender'] = encoded_dataframe['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
    encoded_dataframe = encoded_dataframe.drop(['City'], axis='columns')
    ##########################################################

    return encoded_dataframe


def identify_features_and_targets(encoded_dataframe):
    '''
    Purpose:
    ---
    The purpose of this function is to define the features and
    the required target labels. The function returns a python tuple
    in which the first item is the selected features and second
    item is the target label

    Input Arguments:
    ---
    `encoded_dataframe` : [ Dataframe ]
                        Pandas dataframe that has all the features mapped to
                        numbers starting from zero

    Returns:
    ---
    `features_and_targets` : [ tuple ]
                            python tuple in which the first item is the
                            selected features and second item is the target label

    Example call:
    ---
    features_and_targets = identify_features_and_targets(encoded_dataframe)
    '''

    #################	ADD YOUR CODE HERE	##################
    target_label = encoded_dataframe['LeaveOrNot']
    features = encoded_dataframe.drop('LeaveOrNot', axis=1)
    features_and_targets = ([features, target_label])
    ##########################################################

    return features_and_targets


def load_as_tensors(features_and_targets):
    '''
    Purpose:
    ---
    This function aims at loading your data (both training and validation)
    as PyTorch tensors. Here you will have to split the dataset for training
    and validation, and then load them as as tensors.
    Training of the model requires iterating over the training tensors.
    Hence the training sensors need to be converted to iterable dataset
    object.

    Input Arguments:
    ---
    `features_and targets` : [ tuple ]
                            python tuple in which the first item is the
                            selected features and second item is the target label

    Returns:
    ---
    `tensors_and_iterable_training_data` : [ tuple ]
                                            Items:
                                            [0]: X_train_tensor: Training features loaded into Pytorch array
                                            [1]: X_test_tensor: Feature tensors in validation data
                                            [2]: y_train_tensor: Training labels as Pytorch tensor
                                            [3]: y_test_tensor: Target labels as tensor in validation data
                                            [4]: Iterable dataset object and iterating over it in
                                                 batches, which are then fed into the model for processing

    Example call:
    ---
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)
    '''

    #################	ADD YOUR CODE HERE	##################
    features, target_label = features_and_targets
    X_train, X_test, y_train, y_test = train_test_split(features, target_label, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # Reshape y_train

    # Convert y_test to the appropriate format
    y_test_tensor = torch.FloatTensor(y_test.to_numpy()).view(-1, 1)  # Reshape y_test

    X_test_tensor = torch.FloatTensor(X_test)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    tensors_and_iterable_training_data = (X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader)
    ##########################################################

    return tensors_and_iterable_training_data



class Salary_Predictor(nn.Module):
    '''
    Purpose:
    ---
    The architecture and behavior of your neural network model will be
    defined within this class that inherits from nn.Module. Here you
    also need to specify how the input data is processed through the layers.
    It defines the sequence of operations that transform the input data into
    the predicted output. When an instance of this class is created and data
    is passed through it, the `forward` method is automatically called, and
    the output is the prediction of the model based on the input data.

    Returns:
    ---
    `predicted_output` : Predicted output for the given input data
    '''

    def __init__(self):
        super(Salary_Predictor, self).__init__()
        '''
        Define the type and number of layers
        '''
        #######	ADD YOUR CODE HERE	#######
        self.fc1 = nn.Linear(7, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    ###################################

    def forward(self, predicted_output):
        '''
        Define the activation functions
        '''
        #######	ADD YOUR CODE HERE	#######
        predicted_output = self.fc1(predicted_output)  # Apply the first fully connected layer (linear transformation)
        predicted_output = self.relu(predicted_output)  # Apply the ReLU activation function
        predicted_output = self.fc2(predicted_output)  # Apply the second fully connected layer (linear transformation)
        predicted_output = self.relu(predicted_output)  # Apply the ReLU activation function
        predicted_output = self.fc3(predicted_output)  # Apply the third fully connected layer (linear transformation)
        predicted_output = self.sigmoid(predicted_output)
        ###################################

        return predicted_output


def model_loss_function():
    '''
    Purpose:
    ---
    To define the loss function for the model. Loss function measures
    how well the predictions of a model match the actual target values
    in training data.

    Input Arguments:
    ---
    None

    Returns:
    ---
    `loss_function`: This can be a pre-defined loss function in PyTorch
                    or can be user-defined

    Example call:
    ---
    loss_function = model_loss_function()
    '''
    #################	ADD YOUR CODE HERE	##################

    ##########################################################

    return nn.BCELoss()


def model_optimizer(model):
    '''
    Purpose:
    ---
    To define the optimizer for the model. Optimizer is responsible
    for updating the parameters (weights and biases) in a way that
    minimizes the loss function.

    Input Arguments:
    ---
    `model`: An object of the 'Salary_Predictor' class

    Returns:
    ---
    `optimizer`: Pre-defined optimizer from Pytorch

    Example call:
    ---
    optimizer = model_optimizer(model)
    '''
    #################	ADD YOUR CODE HERE	##################
    #optimizer = optim.Adam(model.parameters(), lr=0.001)
    ##########################################################

    return optim.Adam(model.parameters(), lr=0.001)


def model_number_of_epochs():
    '''
    Purpose:
    ---
    To define the number of epochs for training the model

    Input Arguments:
    ---
    None

    Returns:
    ---
    `number_of_epochs`: [integer value]

    Example call:
    ---
    number_of_epochs = model_number_of_epochs()
    '''
    #################	ADD YOUR CODE HERE	##################
    number_of_epochs = 100
    ##########################################################

    return number_of_epochs


def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    '''
	Purpose:
	---
	All the required parameters for training are passed to this function.

	Input Arguments:
	---
	1. `model`: An object of the 'Salary_Predictor' class
	2. `number_of_epochs`: For training the model
	3. `tensors_and_iterable_training_data`: list containing training and validation data tensors
											 and iterable dataset object of training tensors
	4. `loss_function`: Loss function defined for the model
	5. `optimizer`: Optimizer defined for the model

	Returns:
	---
	trained_model

	Example call:
	---
	trained_model = training_function(model, number_of_epochs, iterable_training_data, loss_function, optimizer)

	'''
    #################	ADD YOUR CODE HERE	##################
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tesnor, train_loader = tensors_and_iterable_training_data
    for epoch in range(number_of_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # it clears the gradients from previous iterations to prevent accumulation.
            outputs = model(
                inputs.float())  # it performs a forward pass through the model, obtaining predictions for the batch of input data.
            outputs = outputs.squeeze(dim=1)
            labels = labels.squeeze(dim=1)
            loss = loss_function(outputs, labels.float())
            loss.backward()  # Backpropagation: compute gradients of the loss
            optimizer.step()  # Update model parameters using the computed gradients
        print(f'Epoch [{epoch + 1}/{number_of_epochs}], Loss: {loss.item():.4f}')
        trained_model = model

    ##########################################################
    return trained_model


def validation_function(trained_model, tensors_and_iterable_training_data):
    '''
    Purpose:
    ---
    This function will utilise the trained model to do predictions on the
    validation dataset. This will enable us to understand the accuracy of
    the model.

    Input Arguments:
    ---
    1. `trained_model`: Returned from the training function
    2. `tensors_and_iterable_training_data`: list containing training and validation data tensors
                                             and iterable dataset object of training tensors

    Returns:
    ---
    model_accuracy: Accuracy on the validation dataset

    Example call:
    ---
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)

    '''
    #################	ADD YOUR CODE HERE	##################
    X_test_tensor, y_test_tensor, val_loader = tensors_and_iterable_training_data[2],tensors_and_iterable_training_data[3], tensors_and_iterable_training_data[4]

    # Initialize all_preds and all_labels
    all_preds = torch.tensor([], dtype=torch.float32)
    all_labels = torch.tensor([], dtype=torch.float32)

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = trained_model(inputs.float())
            predicted = (outputs >= 0.5).float()  # Apply a threshold of 0.5 for binary predictions
            all_preds = torch.cat((all_preds, predicted.view(-1)))  # Concatenate predictions
            all_labels = torch.cat((all_labels, labels.view(-1)))  # Concatenate labels
            print(all_preds)
    model_accuracy = (all_preds == all_labels).float().mean()

    ##########################################################

    return model_accuracy




########################################################################
########################################################################
######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
'''
	Purpose:
	---
	The following is the main function combining all the functions
	mentioned above. Go through this function to understand the flow
	of the script

'''
if __name__ == "__main__":
    # reading the provided dataset csv file using pandas library and
     # converting it to a pandas Dataframe
    task_1a_dataframe = pandas.read_csv("C:/Users/Vedant/Desktop/eyrc/Task_1A/task_1a_dataset.csv")

    # data preprocessing and obtaining encoded data
    encoded_dataframe = data_preprocessing(task_1a_dataframe)

    # selecting required features and targets
    features_and_targets = identify_features_and_targets(encoded_dataframe)

    # obtaining training and validation data tensors and the iterable
    # training data object
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)

    # model is an instance of the class that defines the architecture of the model
    model = Salary_Predictor()

    # obtaining loss function, optimizer and the number of training epochs
    loss_function = model_loss_function()
    optimizer = model_optimizer(model)
    number_of_epochs = model_number_of_epochs()

    # training the model
    trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data,
                                      loss_function, optimizer)

    # validating and obtaining accuracy
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
    print(f"Accuracy on the test set = {model_accuracy}")

    X_train_tensor = tensors_and_iterable_training_data[0]
    x = X_train_tensor[0]
    jitted_model = torch.jit.save(torch.jit.trace(model, (x)), "task_1a_trained_model.pth")



Epoch [1/100], Loss: 0.5471
Epoch [2/100], Loss: 0.4860
Epoch [3/100], Loss: 0.6107
Epoch [4/100], Loss: 0.5017
Epoch [5/100], Loss: 0.5388
Epoch [6/100], Loss: 0.3831
Epoch [7/100], Loss: 0.4537
Epoch [8/100], Loss: 0.4698
Epoch [9/100], Loss: 0.3316
Epoch [10/100], Loss: 0.4564
Epoch [11/100], Loss: 0.3947
Epoch [12/100], Loss: 0.4256
Epoch [13/100], Loss: 0.3853
Epoch [14/100], Loss: 0.3492
Epoch [15/100], Loss: 0.3819
Epoch [16/100], Loss: 0.5441
Epoch [17/100], Loss: 0.4458
Epoch [18/100], Loss: 0.3944
Epoch [19/100], Loss: 0.4159
Epoch [20/100], Loss: 0.6207
Epoch [21/100], Loss: 0.4325
Epoch [22/100], Loss: 0.4121
Epoch [23/100], Loss: 0.2760
Epoch [24/100], Loss: 0.3548
Epoch [25/100], Loss: 0.4991
Epoch [26/100], Loss: 0.3875
Epoch [27/100], Loss: 0.4123
Epoch [28/100], Loss: 0.3298
Epoch [29/100], Loss: 0.4077
Epoch [30/100], Loss: 0.4940
Epoch [31/100], Loss: 0.4498
Epoch [32/100], Loss: 0.4677
Epoch [33/100], Loss: 0.3673
Epoch [34/100], Loss: 0.4100
Epoch [35/100], Loss: 0

# Running code


In [ ]:
'''
*****************************************************************************************
*
*        		===============================================
*           		GeoGuide(GG) Theme (eYRC 2023-24)
*        		===============================================
*
*  This script is to implement Task 1A of GeoGuide(GG) Theme (eYRC 2023-24).
*
*  This software is made available on an "AS IS WHERE IS BASIS".
*  Licensee/end user indemnifies and will keep e-Yantra indemnified from
*  any and all claim(s) that emanate from the use of the Software or
*  breach of the terms of this agreement.
*
*****************************************************************************************
'''

# Team ID:			[ GG_1673]
# Author List:		[ Names of team members worked on this file separated by Comma: Name1, Name2, ... ]
# Filename:			task_1a.py
# Functions:	    [`ideantify_features_and_targets`, `load_as_tensors`,
# 					 `model_loss_function`, `model_optimizer`, `model_number_of_epochs`, `training_function`,
# 					 `validation_functions` ]

####################### IMPORT MODULES #######################
import pandas
import torch
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

###################### Additional Imports ####################
'''
You can import any additional modules that you require from
torch, matplotlib or sklearn.
You are NOT allowed to import any other libraries. It will
cause errors while running the executable
'''


##############################################################

################# ADD UTILITY FUNCTIONS HERE #################


##############################################################

def data_preprocessing(task_1a_dataframe):
    '''
    Purpose:
    ---
    This function will be used to load your csv dataset and preprocess it.
    Preprocessing involves cleaning the dataset by removing unwanted features,
    decision about what needs to be done with missing values etc. Note that
    there are features in the csv file whose values are textual (eg: Industry,
    Education Level etc)These features might be required for training the model
    but can not be given directly as strings for training. Hence this function
    should return encoded dataframe in which all the textual features are
    numerically labeled.

    Input Arguments:
    ---
    `task_1a_dataframe`: [Dataframe]
                          Pandas dataframe read from the provided dataset

    Returns:
    ---
    `encoded_dataframe` : [ Dataframe ]
                          Pandas dataframe that has all the features mapped to
                          numbers starting from zero

    Example call:
    ---
    encoded_dataframe = data_preprocessing(task_1a_dataframe)
    '''

    #################	ADD YOUR CODE HERE	##################
    task_1a_dataframe['Education'] = LabelEncoder().fit_transform(task_1a_dataframe['Education'])
    task_1a_dataframe
    encoded_dataframe = task_1a_dataframe
    encoded_dataframe['EverBenched'] = encoded_dataframe['EverBenched'].apply(lambda x: 1 if x == 'Yes' else 0)
    encoded_dataframe['Gender'] = encoded_dataframe['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
    encoded_dataframe = encoded_dataframe.drop(['City'], axis='columns')
    ##########################################################

    return encoded_dataframe


def identify_features_and_targets(encoded_dataframe):
    '''
    Purpose:
    ---
    The purpose of this function is to define the features and
    the required target labels. The function returns a python list
    in which the first item is the selected features and second
    item is the target label

    Input Arguments:
    ---
    `encoded_dataframe` : [ Dataframe ]
                        Pandas dataframe that has all the features mapped to
                        numbers starting from zero

    Returns:
    ---
    `features_and_targets` : [ list ]
                            python list in which the first item is the
                            selected features and second item is the target label

    Example call:
    ---
    features_and_targets = identify_features_and_targets(encoded_dataframe)
    '''

    #################	ADD YOUR CODE HERE	##################
    target_label = encoded_dataframe['LeaveOrNot']
    features = encoded_dataframe.drop('LeaveOrNot', axis=1)
    features_and_targets = ([features, target_label])
    ##########################################################

    return features_and_targets


def load_as_tensors(features_and_targets):
    '''
    Purpose:
    ---
    This function aims at loading your data (both training and validation)
    as PyTorch tensors. Here you will have to split the dataset for training
    and validation, and then load them as as tensors.
    Training of the model requires iterating over the training tensors.
    Hence the training sensors need to be converted to iterable dataset
    object.

    Input Arguments:
    ---
    `features_and targets` : [ list ]
                            python list in which the first item is the
                            selected features and second item is the target label

    Returns:
    ---
    `tensors_and_iterable_training_data` : [ list ]
                                            Items:
                                            [0]: X_train_tensor: Training features loaded into Pytorch array
                                            [1]: X_test_tensor: Feature tensors in validation data
                                            [2]: y_train_tensor: Training labels as Pytorch tensor
                                            [3]: y_test_tensor: Target labels as tensor in validation data
                                            [4]: Iterable dataset object and iterating over it in
                                                 batches, which are then fed into the model for processing

    Example call:
    ---
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)
    '''

    #################	ADD YOUR CODE HERE	##################
    features, target_label = features_and_targets
    X_train, X_test, y_train, y_test = train_test_split(features, target_label, test_size=0.2, random_state=40)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train.to_numpy()).view(-1, 1)  # Reshape y_train
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.FloatTensor(y_test.to_numpy()).view(-1, 1)  # Reshape y_test
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    tensors_and_iterable_training_data = [X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, train_loader]
    ##########################################################

    return tensors_and_iterable_training_data


class Salary_Predictor(nn.Module):
    '''
    Purpose:
    ---
    The architecture and behavior of your neural network model will be
    defined within this class that inherits from nn.Module. Here you
    also need to specify how the input data is processed through the layers.
    It defines the sequence of operations that transform the input data into
    the predicted output. When an instance of this class is created and data
    is passed through it, the `forward` method is automatically called, and
    the output is the prediction of the model based on the input data.

    Returns:
    ---
    `predicted_output` : Predicted output for the given input data
    '''

    def __init__(self):
        super(Salary_Predictor, self).__init__()
        '''
        Define the type and number of layers
        '''
        #######	ADD YOUR CODE HERE	#######
        self.fc1 = nn.Linear(7, 64)  # 2 input features, 3 output features
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    ###################################

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        predicted_output = x
        return predicted_output


def model_loss_function():
    '''
    Purpose:
    ---
    To define the loss function for the model. Loss function measures
    how well the predictions of a model match the actual target values
    in training data.

    Input Arguments:
    ---
    None

    Returns:
    ---
    `loss_function`: This can be a pre-defined loss function in PyTorch
                    or can be user-defined

    Example call:
    ---
    loss_function = model_loss_function()
    '''
    #################	ADD YOUR CODE HERE	##################

    ##########################################################

    return nn.BCELoss()


def model_optimizer(model):
    '''
    Purpose:
    ---
    To define the optimizer for the model. Optimizer is responsible
    for updating the parameters (weights and biases) in a way that
    minimizes the loss function.

    Input Arguments:
    ---
    `model`: An object of the 'Salary_Predictor' class

    Returns:
    ---
    `optimizer`: Pre-defined optimizer from Pytorch

    Example call:
    ---
    optimizer = model_optimizer(model)
    '''
    #################	ADD YOUR CODE HERE	##################
    #optimizer = optim.Adam(model.parameters(), lr=0.001)
    ##########################################################

    return optim.Adam(model.parameters(), lr=0.001)


def model_number_of_epochs():
    '''
    Purpose:
    ---
    To define the number of epochs for training the model

    Input Arguments:
    ---
    None

    Returns:
    ---
    `number_of_epochs`: [integer value]

    Example call:
    ---
    number_of_epochs = model_number_of_epochs()
    '''
    #################	ADD YOUR CODE HERE	##################
    number_of_epochs = 100
    ##########################################################

    return number_of_epochs


def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    '''
	Purpose:
	---
	All the required parameters for training are passed to this function.

	Input Arguments:
	---
	1. `model`: An object of the 'Salary_Predictor' class
	2. `number_of_epochs`: For training the model
	3. `tensors_and_iterable_training_data`: list containing training and validation data tensors
											 and iterable dataset object of training tensors
	4. `loss_function`: Loss function defined for the model
	5. `optimizer`: Optimizer defined for the model

	Returns:
	---
	trained_model

	Example call:
	---
	trained_model = training_function(model, number_of_epochs, iterable_training_data, loss_function, optimizer)

	'''
    #################	ADD YOUR CODE HERE	##################
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tesnor, train_loader = tensors_and_iterable_training_data
    for epoch in range(number_of_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # it clears the gradients from previous iterations to prevent accumulation.
            outputs = model(inputs.float())  # it performs a forward pass through the model, obtaining predictions for the batch of input data.
            outputs = outputs.squeeze(dim=1)
            labels = labels.squeeze(dim=1)
            loss = loss_function(outputs, labels.float())
            loss.backward()  # Backpropagation: compute gradients of the loss
            optimizer.step()  # Update model parameters using the computed gradients
        print(f'Epoch [{epoch + 1}/{number_of_epochs}], Loss: {loss.item():.4f}')
        trained_model = model
    ##########################################################
    return trained_model


def validation_function(trained_model, tensors_and_iterable_training_data):
    '''
    Purpose:
    ---
    This function will utilise the trained model to do predictions on the
    validation dataset. This will enable us to understand the accuracy of
    the model.

    Input Arguments:
    ---
    1. `trained_model`: Returned from the training function
    2. `tensors_and_iterable_training_data`: list containing training and validation data tensors
                                             and iterable dataset object of training tensors

    Returns:
    ---
    model_accuracy: Accuracy on the validation dataset

    Example call:
    ---
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)

    '''
    #################	ADD YOUR CODE HERE	##################
    X_test_tensor, y_test_tensor, val_loader = tensors_and_iterable_training_data[2], tensors_and_iterable_training_data[3], tensors_and_iterable_training_data[4]
    # X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader = tensors_and_iterable_training_data
    val_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

    trained_model.eval()

    # all_preds = torch.FloatTensor()
    # all_labels = torch.FloatTensor()
    all_preds = torch.tensor([], dtype=torch.float32)
    all_labels = torch.tensor([], dtype=torch.float32)

    # Unpack tensors_and_iterable_training_data
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = trained_model(inputs.float())
            predicted = (outputs >= 0.5).float()  # Apply a threshold of 0.5 for binary predictions
            all_preds = torch.cat((all_preds, predicted.view(-1)))  # Concatenate predictions
            all_labels = torch.cat((all_labels, labels.view(-1)))  # Concatenate labels
    model_accuracy = (all_preds == all_labels).float().mean()

    ##########################################################

    return model_accuracy


########################################################################
########################################################################
######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
'''
	Purpose:
	---
	The following is the main function combining all the functions
	mentioned above. Go through this function to understand the flow
	of the script

'''
if __name__ == "__main__":
    # reading the provided dataset csv file using pandas library and
    # converting it to a pandas Dataframe
    task_1a_dataframe = pandas.read_csv("C:/Users/Vedant/Desktop\eyrc/Task_1A/task_1a_dataset.csv")

    # data preprocessing and obtaining encoded data
    encoded_dataframe = data_preprocessing(task_1a_dataframe)

    # selecting required features and targets
    features_and_targets = identify_features_and_targets(encoded_dataframe)

    # obtaining training and validation data tensors and the iterable
    # training data object
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)

    # model is an instance of the class that defines the architecture of the model
    model = Salary_Predictor()

    # obtaining loss function, optimizer and the number of training epochs
    loss_function = model_loss_function()
    optimizer = model_optimizer(model)
    number_of_epochs = model_number_of_epochs()

    # training the model
    trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data,
                                      loss_function, optimizer)

    # validating and obtaining accuracy
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
    print(f"Accuracy on the test set = {model_accuracy}")

    X_train_tensor = tensors_and_iterable_training_data[0]
    x = X_train_tensor[0]
    jitted_model = torch.jit.save(torch.jit.trace(model, (x)), "task_1a_trained_model.pth")



















# trial

In [ ]:
import pandas
import torch
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

In [ ]:
task_1a_dataframe = pandas.read_csv('/content/task_1a_dataset.csv')
task_1a_dataframe

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2012,Bangalore,3,37,Male,No,0,0
1,Masters,2017,New Delhi,2,28,Male,No,4,0
2,Bachelors,2017,New Delhi,2,36,Male,No,3,0
3,Bachelors,2015,Bangalore,3,27,Male,Yes,5,0
4,Bachelors,2017,Bangalore,3,29,Male,No,4,0
...,...,...,...,...,...,...,...,...,...
4628,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4629,Masters,2013,Pune,2,37,Male,No,2,1
4630,Masters,2018,New Delhi,3,27,Male,No,5,1
4631,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [ ]:
label_encoder = LabelEncoder()
task_1a_dataframe['Education'] = label_encoder.fit_transform(task_1a_dataframe['Education'])
task_1a_dataframe['City'] = label_encoder.fit_transform(task_1a_dataframe['City'])
task_1a_dataframe['PaymentTier'] = label_encoder.fit_transform(task_1a_dataframe['PaymentTier'])
task_1a_dataframe['Gender'] = label_encoder.fit_transform(task_1a_dataframe['Gender'])
encoded_dataframe = task_1a_dataframe
scaler = StandardScaler()
numerical_cols = ['JoiningYear', 'Age', 'ExperienceInCurrentDomain']
encoded_dataframe[numerical_cols] = scaler.fit_transform(encoded_dataframe[numerical_cols])


In [ ]:
encoded_dataframe

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,0,-1.642610,0,2,1.579346,1,No,-1.868200,0
1,1,1.039876,1,1,-0.287422,1,No,0.701112,0
2,0,1.039876,1,1,1.371928,1,No,0.058784,0
3,0,-0.033119,0,2,-0.494841,1,Yes,1.343440,0
4,0,1.039876,0,2,-0.080004,1,No,0.701112,0
...,...,...,...,...,...,...,...,...,...
4628,0,-1.106113,0,2,-0.702260,0,No,0.701112,0
4629,1,-1.106113,2,1,1.579346,1,No,-0.583544,1
4630,1,1.576373,1,2,-0.494841,1,No,1.343440,1
4631,0,-1.642610,0,2,0.127415,1,Yes,-0.583544,0


In [ ]:
task_1a_dataframe['Education'] = LabelEncoder().fit_transform(task_1a_dataframe['Education'])
task_1a_dataframe
encoded_dataframe = task_1a_dataframe
encoded_dataframe['EverBenched'] = encoded_dataframe['EverBenched'].apply(lambda x: 1 if x == 'Yes' else 0)
encoded_dataframe['Gender'] = encoded_dataframe['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
encoded_dataframe = encoded_dataframe.drop(['City'], axis='columns')

In [ ]:
encoded_dataframe

,Education,JoiningYear,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,0,-1.642610,2,1.579346,0,0,-1.868200,0
1,1,1.039876,1,-0.287422,0,0,0.701112,0
2,0,1.039876,1,1.371928,0,0,0.058784,0
3,0,-0.033119,2,-0.494841,0,1,1.343440,0
4,0,1.039876,2,-0.080004,0,0,0.701112,0
...,...,...,...,...,...,...,...,...
4628,0,-1.106113,2,-0.702260,0,0,0.701112,0
4629,1,-1.106113,1,1.579346,0,0,-0.583544,1
4630,1,1.576373,2,-0.494841,0,0,1.343440,1
4631,0,-1.642610,2,0.127415,0,1,-0.583544,0


In [ ]:
target_label = encoded_dataframe['LeaveOrNot']
features = encoded_dataframe.drop('LeaveOrNot', axis=1)
features_and_targets = ([features, target_label])

In [ ]:
features

,Education,JoiningYear,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,0,-1.642610,2,1.579346,0,0,-1.868200
1,1,1.039876,1,-0.287422,0,0,0.701112
2,0,1.039876,1,1.371928,0,0,0.058784
3,0,-0.033119,2,-0.494841,0,1,1.343440
4,0,1.039876,2,-0.080004,0,0,0.701112
...,...,...,...,...,...,...,...
4628,0,-1.106113,2,-0.702260,0,0,0.701112
4629,1,-1.106113,1,1.579346,0,0,-0.583544
4630,1,1.576373,2,-0.494841,0,0,1.343440
4631,0,-1.642610,2,0.127415,0,1,-0.583544


In [ ]:
target_label

0       0
1       0
2       0
3       0
4       0
       ..
4628    0
4629    1
4630    1
4631    0
4632    0
Name: LeaveOrNot, Length: 4633, dtype: int64

In [ ]:
features_and_targets

[      Education  JoiningYear  PaymentTier       Age  Gender  EverBenched  \
 0             0    -1.642610            2  1.579346       0            0   
 1             1     1.039876            1 -0.287422       0            0   
 2             0     1.039876            1  1.371928       0            0   
 3             0    -0.033119            2 -0.494841       0            1   
 4             0     1.039876            2 -0.080004       0            0   
 ...         ...          ...          ...       ...     ...          ...   
 4628          0    -1.106113            2 -0.702260       0            0   
 4629          1    -1.106113            1  1.579346       0            0   
 4630          1     1.576373            2 -0.494841       0            0   
 4631          0    -1.642610            2  0.127415       0            1   
 4632          0    -0.033119            2  0.749671       0            1   
 
       ExperienceInCurrentDomain  
 0                     -1.868200  
 1  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target_label, test_size=0.2, random_state=42)

In [ ]:
print(X_train)

      Education  JoiningYear  PaymentTier       Age  Gender  EverBenched  \
881           0     1.039876            2 -1.117098       0            0   
1501          2     0.503379            2 -0.494841       0            0   
1204          0     0.503379            2 -0.702260       0            0   
2223          2    -0.569616            1 -0.494841       0            0   
4018          1     1.576373            2  1.994184       0            0   
...         ...          ...          ...       ...     ...          ...   
4426          0     1.039876            2 -0.702260       0            0   
466           0    -0.033119            1 -0.909679       0            1   
3092          0     0.503379            2  1.994184       0            0   
3772          1     0.503379            2  1.579346       0            0   
860           0     1.039876            2 -0.494841       0            0   

      ExperienceInCurrentDomain  
881                   -0.583544  
1501               

In [ ]:
X_test

,Education,JoiningYear,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
805,0,-1.642610,2,-0.494841,0,0,1.343440
1820,1,1.039876,1,-0.702260,0,0,0.701112
3617,0,1.039876,2,1.994184,0,0,0.058784
2685,0,-1.106113,2,-0.702260,0,0,0.701112
33,0,-0.569616,2,0.334834,0,1,1.343440
...,...,...,...,...,...,...,...
3153,0,-1.106113,2,2.409021,0,0,-0.583544
360,0,0.503379,2,-0.287422,0,0,-0.583544
1253,1,-1.106113,2,-0.287422,0,0,-0.583544
3106,0,-0.569616,2,1.164509,0,0,0.058784


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train


array([[-0.5016399 , -1.6409554 ,  0.53383058, ...,  0.        ,
         0.        ,  1.34983167],
       [ 1.42088074,  1.04445514, -1.25332039, ...,  0.        ,
         0.        ,  0.70791249],
       [-0.5016399 ,  1.04445514,  0.53383058, ...,  0.        ,
         0.        ,  0.06599331],
       ...,
       [ 1.42088074, -1.10387329,  0.53383058, ...,  0.        ,
         0.        , -0.57592587],
       [-0.5016399 , -0.56679118,  0.53383058, ...,  0.        ,
         0.        ,  0.06599331],
       [-0.5016399 ,  1.58153725,  0.53383058, ...,  0.        ,
         0.        , -1.21784505]])

In [ ]:
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # Reshape y_train
# Convert y_test to the appropriate format
y_test_tensor = torch.FloatTensor(y_test.to_numpy()).view(-1, 1)  # Reshape y_test
X_test_tensor = torch.FloatTensor(X_test)

In [ ]:
X_train_tensor

tensor([[-0.5016,  1.0445,  0.5338,  ...,  0.0000,  0.0000, -0.5759],
        [ 3.3434,  0.5074,  0.5338,  ...,  0.0000,  0.0000,  1.3498],
        [-0.5016,  0.5074,  0.5338,  ...,  0.0000,  0.0000,  0.7079],
        ...,
        [-0.5016,  0.5074,  0.5338,  ...,  0.0000,  0.0000,  1.3498],
        [ 1.4209,  0.5074,  0.5338,  ...,  0.0000,  0.0000,  0.0660],
        [-0.5016,  1.0445,  0.5338,  ...,  0.0000,  0.0000,  1.3498]])

# New one


In [ ]:

'''
*****************************************************************************************
*
*        		===============================================
*           		GeoGuide(GG) Theme (eYRC 2023-24)
*        		===============================================
*
*  This script is to implement Task 1A of GeoGuide(GG) Theme (eYRC 2023-24).
*
*  This software is made available on an "AS IS WHERE IS BASIS".
*  Licensee/end user indemnifies and will keep e-Yantra indemnified from
*  any and all claim(s) that emanate from the use of the Software or
*  breach of the terms of this agreement.
*
*****************************************************************************************
'''

# Team ID:			[ GG_1673 ]
# Author List:		[ Vedant Adulkar, Dhanashree Pawar, Atharva Kocharekar, Mohammed Hazique Sayed]
# Filename:			task_1a.py
# Functions:	    [`ideantify_features_and_targets`, `load_as_tensors`,
# 					 `model_loss_function`, `model_optimizer`, `model_number_of_epochs`, `training_function`,
# 					 `validation_functions` ]

####################### IMPORT MODULES #######################
import pandas
import torch
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

###################### Additional Imports ####################
'''
You can import any additional modules that you require from
torch, matplotlib or sklearn.
You are NOT allowed to import any other libraries. It will
cause errors while running the executable
'''


##############################################################

################# ADD UTILITY FUNCTIONS HERE #################


##############################################################

def data_preprocessing(task_1a_dataframe):
    '''
    Purpose:
    ---
    This function will be used to load your csv dataset and preprocess it.
    Preprocessing involves cleaning the dataset by removing unwanted features,
    decision about what needs to be done with missing values etc. Note that
    there are features in the csv file whose values are textual (eg: Industry,
    Education Level etc)These features might be required for training the model
    but can not be given directly as strings for training. Hence this function
    should return encoded dataframe in which all the textual features are
    numerically labeled.

    Input Arguments:
    ---
    `task_1a_dataframe`: [Dataframe]
                          Pandas dataframe read from the provided dataset

    Returns:
    ---
    `encoded_dataframe` : [ Dataframe ]
                          Pandas dataframe that has all the features mapped to
                          numbers starting from zero

    Example call:
    ---
    encoded_dataframe = data_preprocessing(task_1a_dataframe)
    '''

    #################	ADD YOUR CODE HERE	##################
    task_1a_dataframe['Education'] = LabelEncoder().fit_transform(task_1a_dataframe['Education'])
    encoded_dataframe = task_1a_dataframe
    encoded_dataframe['EverBenched'] = encoded_dataframe['EverBenched'].apply(lambda x: 1 if x == 'Yes' else 0)
    encoded_dataframe['Gender'] = encoded_dataframe['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
    encoded_dataframe = encoded_dataframe.drop(['City'], axis='columns')
    ##########################################################

    return encoded_dataframe


def identify_features_and_targets(encoded_dataframe):
    '''
    Purpose:
    ---
    The purpose of this function is to define the features and
    the required target labels. The function returns a python tuple
    in which the first item is the selected features and second
    item is the target label

    Input Arguments:
    ---
    `encoded_dataframe` : [ Dataframe ]
                        Pandas dataframe that has all the features mapped to
                        numbers starting from zero

    Returns:
    ---
    `features_and_targets` : [ tuple ]
                            python tuple in which the first item is the
                            selected features and second item is the target label

    Example call:
    ---
    features_and_targets = identify_features_and_targets(encoded_dataframe)
    '''

    #################	ADD YOUR CODE HERE	##################
    target_label = encoded_dataframe['LeaveOrNot']
    features = encoded_dataframe.drop('LeaveOrNot', axis=1)
    features_and_targets = ([features, target_label])
    ##########################################################

    return features_and_targets


def load_as_tensors(features_and_targets):
    '''
    Purpose:
    ---
    This function aims at loading your data (both training and validation)
    as PyTorch tensors. Here you will have to split the dataset for training
    and validation, and then load them as as tensors.
    Training of the model requires iterating over the training tensors.
    Hence the training sensors need to be converted to iterable dataset
    object.

    Input Arguments:
    ---
    `features_and targets` : [ tuple ]
                            python tuple in which the first item is the
                            selected features and second item is the target label

    Returns:
    ---
    `tensors_and_iterable_training_data` : [ tuple ]
                                            Items:
                                            [0]: X_train_tensor: Training features loaded into Pytorch array
                                            [1]: X_test_tensor: Feature tensors in validation data
                                            [2]: y_train_tensor: Training labels as Pytorch tensor
                                            [3]: y_test_tensor: Target labels as tensor in validation data
                                            [4]: Iterable dataset object and iterating over it in
                                                 batches, which are then fed into the model for processing

    Example call:
    ---
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)
    '''

    #################	ADD YOUR CODE HERE	##################
    features, target_label = features_and_targets
    X_train, X_test, y_train, y_test = train_test_split(features, target_label, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # Reshape y_train

    # Convert y_test to the appropriate format
    y_test_tensor = torch.FloatTensor(y_test.to_numpy()).view(-1, 1)  # Reshape y_test

    X_test_tensor = torch.FloatTensor(X_test)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    tensors_and_iterable_training_data = (X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader)
    ##########################################################

    return tensors_and_iterable_training_data



class Salary_Predictor(nn.Module):
    '''
    Purpose:
    ---
    The architecture and behavior of your neural network model will be
    defined within this class that inherits from nn.Module. Here you
    also need to specify how the input data is processed through the layers.
    It defines the sequence of operations that transform the input data into
    the predicted output. When an instance of this class is created and data
    is passed through it, the `forward` method is automatically called, and
    the output is the prediction of the model based on the input data.

    Returns:
    ---
    `predicted_output` : Predicted output for the given input data
    '''

    def __init__(self):
        super(Salary_Predictor, self).__init__()
        '''
        Define the type and number of layers
        '''
        #######	ADD YOUR CODE HERE	#######
        self.fc1 = nn.Linear(7, 64)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    ###################################

    def forward(self, x):
        '''
        Define the activation functions
        '''
        #######	ADD YOUR CODE HERE	#######
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        predicted_output = x
        ###################################

        return predicted_output


def model_loss_function():
    '''
    Purpose:
    ---
    To define the loss function for the model. Loss function measures
    how well the predictions of a model match the actual target values
    in training data.

    Input Arguments:
    ---
    None

    Returns:
    ---
    `loss_function`: This can be a pre-defined loss function in PyTorch
                    or can be user-defined

    Example call:
    ---
    loss_function = model_loss_function()
    '''
    #################	ADD YOUR CODE HERE	##################

    ##########################################################

    return nn.BCELoss()


def model_optimizer(model):
    '''
    Purpose:
    ---
    To define the optimizer for the model. Optimizer is responsible
    for updating the parameters (weights and biases) in a way that
    minimizes the loss function.

    Input Arguments:
    ---
    `model`: An object of the 'Salary_Predictor' class

    Returns:
    ---
    `optimizer`: Pre-defined optimizer from Pytorch

    Example call:
    ---
    optimizer = model_optimizer(model)
    '''
    #################	ADD YOUR CODE HERE	##################
    #optimizer = optim.Adam(model.parameters(), lr=0.001)
    ##########################################################

    return optim.Adam(model.parameters(), lr=0.001)


def model_number_of_epochs():
    '''
    Purpose:
    ---
    To define the number of epochs for training the model

    Input Arguments:
    ---
    None

    Returns:
    ---
    `number_of_epochs`: [integer value]

    Example call:
    ---
    number_of_epochs = model_number_of_epochs()
    '''
    #################	ADD YOUR CODE HERE	##################
    number_of_epochs = 50
    ##########################################################

    return number_of_epochs


def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    '''
	Purpose:
	---
	All the required parameters for training are passed to this function.

	Input Arguments:
	---
	1. `model`: An object of the 'Salary_Predictor' class
	2. `number_of_epochs`: For training the model
	3. `tensors_and_iterable_training_data`: list containing training and validation data tensors
											 and iterable dataset object of training tensors
	4. `loss_function`: Loss function defined for the model
	5. `optimizer`: Optimizer defined for the model

	Returns:
	---
	trained_model

	Example call:
	---
	trained_model = training_function(model, number_of_epochs, iterable_training_data, loss_function, optimizer)

	'''
    #################	ADD YOUR CODE HERE	##################
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader = tensors_and_iterable_training_data
    for epoch in range(number_of_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            train_outputs = model(X_train_tensor)
            train_predictions = (train_outputs >= 0.5).float()
            train_accuracy = torch.mean((train_predictions == y_train_tensor).float()).item()
            print(f'Epoch {epoch + 1}/{number_of_epochs}, Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%')
    trained_model = model
    ##########################################################
    return trained_model


def validation_function(trained_model, tensors_and_iterable_training_data):
    '''
    Purpose:
    ---
    This function will utilise the trained model to do predictions on the
    validation dataset. This will enable us to understand the accuracy of
    the model.

    Input Arguments:
    ---
    1. `trained_model`: Returned from the training function
    2. `tensors_and_iterable_training_data`: list containing training and validation data tensors
                                             and iterable dataset object of training tensors

    Returns:
    ---
    model_accuracy: Accuracy on the validation dataset

    Example call:
    ---
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)

    '''
    #################	ADD YOUR CODE HERE	##################
    X_test_tensor, y_test_tensor, val_loader = tensors_and_iterable_training_data[1], tensors_and_iterable_training_data[3], tensors_and_iterable_training_data[4]

    with torch.no_grad():
        test_outputs = trained_model(X_test_tensor)
        test_predictions = (test_outputs >= 0.5).float()
        test_accuracy = torch.mean((test_predictions == y_test_tensor).float()).item()
        print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

    model_accuracy = test_accuracy

    #model_accuracy = (all_preds == all_labels).float().mean()

    ##########################################################

    return model_accuracy




########################################################################
########################################################################
######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
'''
	Purpose:
	---
	The following is the main function combining all the functions
	mentioned above. Go through this function to understand the flow
	of the script

'''
if __name__ == "__main__":
    # reading the provided dataset csv file using pandas library and
     # converting it to a pandas Dataframe
    task_1a_dataframe = pandas.read_csv("/content/task_1a_dataset.csv")

    # data preprocessing and obtaining encoded data
    encoded_dataframe = data_preprocessing(task_1a_dataframe)

    # selecting required features and targets
    features_and_targets = identify_features_and_targets(encoded_dataframe)

    # obtaining training and validation data tensors and the iterable
    # training data object
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)

    # model is an instance of the class that defines the architecture of the model
    model = Salary_Predictor()

    # obtaining loss function, optimizer and the number of training epochs
    loss_function = model_loss_function()
    optimizer = model_optimizer(model)
    number_of_epochs = model_number_of_epochs()

    # training the model
    trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data,
                                      loss_function, optimizer)



    # validating and obtaining accuracy
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
    #print(f"Accuracy on the test set = {model_accuracy}")

    X_train_tensor = tensors_and_iterable_training_data[0]
    x = X_train_tensor[0]
    jitted_model = torch.jit.save(torch.jit.trace(model, (x)), "task_1a_trained_model.pth")



Epoch 1/50, Loss: 0.5235, Training Accuracy: 66.62%
Epoch 2/50, Loss: 0.5737, Training Accuracy: 71.21%
Epoch 3/50, Loss: 0.5195, Training Accuracy: 72.32%
Epoch 4/50, Loss: 0.5246, Training Accuracy: 73.29%
Epoch 5/50, Loss: 0.4937, Training Accuracy: 75.42%
Epoch 6/50, Loss: 0.4653, Training Accuracy: 75.01%
Epoch 7/50, Loss: 0.4433, Training Accuracy: 76.63%
Epoch 8/50, Loss: 0.4610, Training Accuracy: 76.77%
Epoch 9/50, Loss: 0.4527, Training Accuracy: 77.77%
Epoch 10/50, Loss: 0.4532, Training Accuracy: 78.14%
Epoch 11/50, Loss: 0.7287, Training Accuracy: 78.17%
Epoch 12/50, Loss: 0.3955, Training Accuracy: 78.31%
Epoch 13/50, Loss: 0.4288, Training Accuracy: 78.79%
Epoch 14/50, Loss: 0.4075, Training Accuracy: 78.60%
Epoch 15/50, Loss: 0.4264, Training Accuracy: 79.47%
Epoch 16/50, Loss: 0.4676, Training Accuracy: 79.57%
Epoch 17/50, Loss: 0.4674, Training Accuracy: 79.47%
Epoch 18/50, Loss: 0.5808, Training Accuracy: 79.65%
Epoch 19/50, Loss: 0.5171, Training Accuracy: 79.87%
Ep

/usr/local/lib/python3.10/dist-packages/torch/jit/_trace.py:1084: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1 / 1 (100.0%)
Greatest absolute difference: 0.008261129260063171 at index (0,) (up to 1e-05 allowed)
Greatest relative difference: 0.0773038666853235 at index (0,) (up to 1e-05 allowed)
  _check_trace(


# New Section

In [ ]:
'''
*******************************
*
*        		===============================================
*           		GeoGuide(GG) Theme (eYRC 2023-24)
*        		===============================================
*
*  This script is to implement Task 1A of GeoGuide(GG) Theme (eYRC 2023-24).
*
*  This software is made available on an "AS IS WHERE IS BASIS".
*  Licensee/end user indemnifies and will keep e-Yantra indemnified from
*  any and all claim(s) that emanate from the use of the Software or
*  breach of the terms of this agreement.
*
*******************************
'''

# Team ID:			[ GG_1673 ]
# Author List:		[ Vedant Adulkar, Dhanashree Pawar, Atharva Kocharekar, Mohammed Hazique Sayed]
# Filename:			task_1a.py
# Functions:	    [`ideantify_features_and_targets`, `load_as_tensors`,
# 					 `model_loss_function`, `model_optimizer`, `model_number_of_epochs`, `training_function`,
# 					 `validation_functions` ]

####################### IMPORT MODULES #######################
import pandas
import torch
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


def data_preprocessing(task_1a_dataframe):
    task_1a_dataframe['Education'] = LabelEncoder().fit_transform(task_1a_dataframe['Education'])
    encoded_dataframe = task_1a_dataframe
    encoded_dataframe['EverBenched'] = encoded_dataframe['EverBenched'].apply(lambda x: 1 if x == 'Yes' else 0)
    encoded_dataframe['Gender'] = encoded_dataframe['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
    encoded_dataframe = encoded_dataframe.drop(['City', 'Gender', 'JoiningYear', 'PaymentTier'], axis='columns')

    return encoded_dataframe

    ##########################################################


def identify_features_and_targets(encoded_dataframe):
    target_label = encoded_dataframe['LeaveOrNot']
    features = encoded_dataframe.drop('LeaveOrNot', axis=1)
    features_and_targets = ([features, target_label])

    return features_and_targets

    ##########################################################


def load_as_tensors(features_and_targets):
    features, target_label = features_and_targets
    X_train, X_test, y_train, y_test = train_test_split(features, target_label, test_size=0.3, random_state=10)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # Reshape y_train

    # Convert y_test to the appropriate format
    y_test_tensor = torch.FloatTensor(y_test.to_numpy()).view(-1, 1)  # Reshape y_test

    X_test_tensor = torch.FloatTensor(X_test)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    tensors_and_iterable_training_data = (X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader)

    return tensors_and_iterable_training_data

    ##########################################################


class Salary_Predictor(nn.Module):

    def __init__(self):
        super(Salary_Predictor, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        # self.fc4 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    ###################################

    def forward(self, predicted_output):
        predicted_output = self.fc1(predicted_output)  # Apply the first fully connected layer (linear transformation)
        predicted_output = self.relu(predicted_output)  # Apply the ReLU activation function
        predicted_output = self.fc2(predicted_output)  # Apply the second fully connected layer (linear transformation)
        predicted_output = self.relu(predicted_output)  # Apply the ReLU activation function
        predicted_output = self.fc3(predicted_output)  # Apply the third fully connected layer (linear transformation)
        # predicted_output = self.relu(predicted_output)  # Apply the ReLU activation function
        # predicted_output = self.fc4(predicted_output)  # Apply the third fully connected layer (linear transformation)
        predicted_output = self.sigmoid(predicted_output)

        return predicted_output

    ##########################################################


def model_loss_function():
    return nn.BCELoss()

    ##########################################################


def model_optimizer(model):
    optimizer = optim.Adam(model.parameters(), lr=0.009)
    return optimizer

    ##########################################################


def model_number_of_epochs():
    number_of_epochs = 100

    return number_of_epochs

    ##########################################################


def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader = tensors_and_iterable_training_data
    for epoch in range(number_of_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # it clears the gradients from previous iterations to prevent accumulation.
            outputs = model(
                inputs.float())  # it performs a forward pass through the model, obtaining predictions for the batch of input data.
            outputs = outputs.squeeze(dim=1)
            labels = labels.squeeze(dim=1)
            loss = loss_function(outputs, labels.float())
            loss.backward()  # Backpropagation: compute gradients of the loss
            optimizer.step()  # Update model parameters using the computed gradients
            # outputs = torch.round(outputs)
        print(f'Epoch [{epoch + 1}/{number_of_epochs}], Loss: {loss.item():.4f}')
        trained_model = model

    return trained_model

    ##########################################################


def validation_function(trained_model, tensors_and_iterable_training_data):
    X_test_tensor, y_test_tensor = tensors_and_iterable_training_data[1], tensors_and_iterable_training_data[3]
    # val_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    # val_loader = DataLoader(val_dataset, batch_size=70, shuffle=True)
    # Initialize all_preds and all_labels
    all_preds = torch.tensor([], dtype=torch.float32)
    all_labels = torch.tensor([], dtype=torch.float32)

    with torch.no_grad():
        # for inputs, labels in val_loader:
        outputs = trained_model(X_test_tensor)  # outputs = trained_model(inputs.float())
        predicted = torch.round(outputs)
        predicted = (outputs >= 0.5).float()  # Apply a threshold of 0.5 for binary predictions
        all_preds = torch.cat((all_preds, predicted.view(-1)))  # Concatenate predictions
        all_labels = torch.cat((all_labels, y_test_tensor.view(-1)))  # Concatenate labels
        print('Labels:', all_labels.shape)
        print('\npreds:', all_preds.shape)
    model_accuracy = accuracy_score(all_labels, all_preds)

    return model_accuracy


########################################################################
########################################################################
######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
'''
	Purpose:
	---
	The following is the main function combining all the functions
	mentioned above. Go through this function to understand the flow
	of the script

'''
if __name__ == "__main__":
    # reading the provided dataset csv file using pandas library and
    # converting it to a pandas Dataframe
    task_1a_dataframe = pandas.read_csv("C:/Users/DP/Desktop/Task_1A/task_1a_dataset.csv")

    # data preprocessing and obtaining encoded data
    encoded_dataframe = data_preprocessing(task_1a_dataframe)

    # selecting required features and targets
    features_and_targets = identify_features_and_targets(encoded_dataframe)

    # obtaining training and validation data tensors and the iterable
    # training data object
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)

    # model is an instance of the class that defines the architecture of the model
    model = Salary_Predictor()

    # obtaining loss function, optimizer and the number of training epochs
    loss_function = model_loss_function()
    optimizer = model_optimizer(model)
    number_of_epochs = model_number_of_epochs()

    # training the model
    trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data,
                                      loss_function, optimizer)

    # validating and obtaining accuracy
    model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
    print(f"Accuracy on the test set = {model_accuracy}")

    X_train_tensor = tensors_and_iterable_training_data[0]
    x = X_train_tensor[0]
    jitted_model = torch.jit.save(torch.jit.trace(model, (x)), "task_1a_trained_model.pth")
